In [6]:
import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import ast

In [2]:
df = pd.read_csv("pre_processed.csv", on_bad_lines='skip')


/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_10064/477583198.py:1: DtypeWarning: Columns (2,7,9,14,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("pre_processed.csv", on_bad_lines='skip')


In [3]:
df.fillna('', inplace=True)

/var/folders/kx/vwf66kn17s153dgnqfn48k180000gn/T/ipykernel_10064/1272664119.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [4]:
def list_to_sentence(list_str):
    word_list = ast.literal_eval(list_str)
    return ' '.join(word_list)

In [7]:
df['self_text_tokens'] = df['self_text_tokens'].apply(list_to_sentence)

In [8]:
df['post_self_text_tokens'] = df['post_self_text_tokens'].apply(list_to_sentence)

In [9]:
df['post_title_tokens'] = df['post_title_tokens'].apply(list_to_sentence)

In [10]:
df['text'] = df['self_text_tokens'].fillna('') + ' ' + df['post_self_text_tokens'].fillna('') + ' ' + df['post_title_tokens'].fillna('')

In [11]:
df["text"].iloc[1]

'if only there was armed military force nearby able to keep law and order if only there wa armed military force nearby able to keep law and order nan nan local gaza gangs are robbing displaced palestinians local gaza gang are robbing displaced palestinian'

In [12]:
df.head()

,created_time,comment_id,score,self_text,subreddit,post_id,author_name,controversiality,ups,user_is_verified,...,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_created_time,post_title_tokens,self_text_tokens,post_self_text_tokens,text
0,2024-03-19 19:18:08,kvmer6g,1,What is this supposed to prove? That nobody in...,IsraelPalestine,1biqebd,tinamnstrrr,0,1.0,True,...,0.0,"So, Mr. Netanyahu, and supporters, are you goo...",article: Famine in northern Gaza is imminent a...,0.36,0,2024-03-19 17:48:02,article famine in northern gaza is imminent mo...,is supposed to prove nobody in gaza has any wa...,so netanyahu and supporters are you good with ...,is supposed to prove nobody in gaza has any wa...
1,2024-03-19 19:17:51,kvmepe5,1,If only there was an armed military force near...,worldnews,1bik6xp,seclifered,0,1.0,True,...,685.0,,Local Gaza gangs are ‘robbing displaced Palest...,0.89,685,2024-03-19 13:25:00,local gaza gangs are robbing displaced palesti...,if only there was armed military force nearby ...,nan nan,if only there was armed military force nearby ...
2,2024-03-19 19:17:35,kvmenpq,1,Taliban created and funded by america and isis...,IsraelPalestine,1biqbsz,Accomplished_Mud6174,0,1.0,True,...,14.0,Most pro Palestine supporters who are not Arab...,Why is the West so concerned?,0.75,14,2024-03-19 17:45:08,why is west so concerned why is west so concerned,taliban created and funded by america and isis...,most pro palestine supporters who are not arab...,taliban created and funded by america and isis...
3,2024-03-19 19:17:25,kvmemm5,1,Recycling glass is still not economical or par...,worldnews,1biqh0j,lordraiden007,0,1.0,True,...,519.0,,The plastic industry knowingly pushed recyclin...,0.97,519,2024-03-19 17:51:10,plastic industry knowingly pushed recycling my...,recycling glass is still economical or particu...,nan nan,recycling glass is still economical or particu...
4,2024-03-19 19:17:22,kvmem94,1,The two have a private direct communication li...,worldnews,1bis6xt,MajorTechnology8827,0,1.0,True,...,0.0,,Biden privately told Netanyahu he's not trying...,0.5,0,2024-03-19 19:00:41,biden privately told netanyahu he not trying t...,two a private direct communication line if any...,nan nan,two a private direct communication line if any...


In [13]:
col_selected = ['created_time','post_created_time', 'subreddit','text']
df = df[col_selected]

In [14]:
import pandas as pd
from textblob import TextBlob

# Apply TextBlob for sentiment analysis
post_sentiment_scores = df['text'].apply(lambda x: TextBlob(str(x)).sentiment)

# Extract polarity scores and subjectivity scores
df['text_polarity'] = post_sentiment_scores.apply(lambda x: x.polarity)
df['text_subjectivity'] = post_sentiment_scores.apply(lambda x: x.subjectivity)

# Assign sentiment category based on sentiment score
df['text_category'] = df['text_polarity'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))

# Count the occurrences of each sentiment category
sentiment_counts = df['text_category'].value_counts()
sentiment_counts_df = sentiment_counts.reset_index()
sentiment_counts_df.columns = ['text_polarity', 'Count']
sentiment_counts_df = sentiment_counts_df.sort_values(by='Count', ascending=False)
sentiment_counts_df.style.background_gradient(cmap='YlGnBu')


,Post_Sentiment,Count
0,Positive,604836
1,Negative,313309
2,Neutral,169692


In [15]:
sentiment_counts_df.columns = ['text_polarity', 'Count']
sentiment_counts_df = sentiment_counts_df.sort_values(by='Count', ascending=False)
sentiment_counts_df.style.background_gradient(cmap='YlGnBu')

,text_polarity,Count
0,Positive,604836
1,Negative,313309
2,Neutral,169692


In [29]:
df.to_csv('Senti_Pre_Processed.csv', index=False)

In [27]:
pip install --upgrade nbformat

Python(13467) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
